In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Optional
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
#1 create stats
class EssayInput(TypedDict):
    eassy: str # clarity of thought
    cot_score: Optional[float]
    depth_of_analysis_score: Optional[float]
    language_proficiency_score: Optional[float]
    average_score: Optional[float]

class EssayOutput(BaseModel):
    score: float = Field(..., description="The overall score of the essay out of 10.")
    
#2 create graph with states
graph = StateGraph(EssayInput)    
    

In [3]:
#2 create llm instance
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
parser = PydanticOutputParser(pydantic_object=EssayOutput)

In [4]:
#3 create nodes function
def cot(state: EssayInput):
    prompt = PromptTemplate(
        template="""
hi you are an expert in UpscEssy . i will provide you an essay evaluation based on clarity of thought.
Please provide a score out of 10 based on the clarity of thought in the essay.
Essay Evaluation: {eassy}
{format_instructions}
""",
        input_variables=["eassy"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )
    chain = prompt | llm | parser
    result = chain.invoke({
        "eassy": state["eassy"]
    })
    state["cot_score"] = result.score
    return {"cot_score": state['cot_score']}

def depth_of_analysis(state: EssayInput):
    prompt = PromptTemplate(
        template="""
hi you are an expert in UpscEssy . i will provide you an essay evaluation based on depth of analysis.
Please provide a score out of 10 based on the depth of analysis in the essay. 
Essay Evaluation: {eassy}
{format_instructions}""",
input_variables=['eassy'],
partial_variables={"format_instructions": parser.get_format_instructions()},
    )
    chain = prompt | llm | parser
    result = chain.invoke({
        "eassy": state["eassy"]
    })
    state["depth_of_analysis_score"] = result.score
    return {'depth_of_analysis_score': state['depth_of_analysis_score']}

def language_proficiency(state: EssayInput):
    prompt = PromptTemplate(
        template="""
hi you are an expert in UpscEssy . i will provide you an essay evaluation based on language proficiency.
Please provide a score out of 10 based on the language proficiency in the essay.
Essay Evaluation: {eassy}
{format_instructions}""",
input_variables=['eassy'],
partial_variables={"format_instructions": parser.get_format_instructions()},
    )
    chain = prompt | llm | parser
    result = chain.invoke({
        "eassy": state["eassy"]
    })
    state["language_proficiency_score"] = result.score
    return {'language_proficiency_score': state['language_proficiency_score']}

def average_score(state: EssayInput):
    cot_score = state.get("cot_score", 0)
    depth_of_analysis_score = state.get("depth_of_analysis_score", 0)
    language_proficiency_score = state.get("language_proficiency_score", 0)
    average = (cot_score + depth_of_analysis_score + language_proficiency_score) / 3
    state["average_score"] = average
    return state


    

In [5]:
#create nodes

graph.add_node("clarityOfThought", cot)
graph.add_node("depth_of_analysis_node", depth_of_analysis)
graph.add_node("language", language_proficiency)
graph.add_node("avg", average_score)


#add edges
graph.add_edge(START, "clarityOfThought")
graph.add_edge(START, "depth_of_analysis_node")
graph.add_edge(START, "language")

graph.add_edge("clarityOfThought", "avg")
graph.add_edge("depth_of_analysis_node", "avg")
graph.add_edge("language", "avg")

graph.add_edge("avg", END)


In [8]:
workflow = graph.compile()
result = workflow.invoke({
    "eassy": "Human civilization has always advanced on the pillars of innovation. From the discovery of fire to the invention of the wheel, from industrial machinery to artificial intelligence, technology has consistently shaped the trajectory of human progress. Yet, every leap forward has confronted humanity with new moral dilemmas. Today, as the world enters an era defined by artificial intelligence, biotechnology, quantum computing, and space exploration, the gap between what humanity can do and what it should do has never been wider. The future of humanity will not be determined solely by the pace of technological progress, but by our ability to anchor that progress in ethical responsibility."
})

In [9]:
result

{'eassy': 'Human civilization has always advanced on the pillars of innovation. From the discovery of fire to the invention of the wheel, from industrial machinery to artificial intelligence, technology has consistently shaped the trajectory of human progress. Yet, every leap forward has confronted humanity with new moral dilemmas. Today, as the world enters an era defined by artificial intelligence, biotechnology, quantum computing, and space exploration, the gap between what humanity can do and what it should do has never been wider. The future of humanity will not be determined solely by the pace of technological progress, but by our ability to anchor that progress in ethical responsibility.',
 'cot_score': 9.5,
 'depth_of_analysis_score': 8.5,
 'language_proficiency_score': 10.0,
 'average_score': 9.333333333333334}